# Linear Solution

In [1]:
from __future__ import print_function

In [2]:
from salib import extend, import_notebooks
import_notebooks()
import numpy as np
from collections import defaultdict

In [3]:
from Frame2D_Base import Frame2D
import Frame2D_Input
import Frame2D_Display

Compiling notebook 'Frame2D_Display.ipynb' to 'Frame2D_Display.py'.


In [4]:
##test:
f = Frame2D('frame-1')
f.install_all()

## Analysis

In [5]:
@extend
class Frame2D:
    
    def buildK(self):
        K = np.mat(np.zeros((self.ndof,self.ndof)))
        for memb in self.members.values():
            Kl = memb.localK()
            Tm = memb.transform()
            Kg = Tm.T * Kl * Tm
            dofnums = memb.nodej.dofnums + memb.nodek.dofnums
            K[np.ix_(dofnums,dofnums)] += Kg
        return K


In [6]:
##test:
K = f.buildK()
K

matrix([[  3.39581250e+05,   0.00000000e+00,   0.00000000e+00,
          -3.37500000e+05,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,  -2.08125000e+03,   0.00000000e+00,
           8.32500000e+06,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   6.17287500e+05,   9.15000000e+06,
           0.00000000e+00,  -2.28750000e+03,   9.15000000e+06,
           0.00000000e+00,   0.00000000e+00,  -6.15000000e+05,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   9.15000000e+06,   4.88000000e+10,
           0.00000000e+00,  -9.15000000e+06,   2.44000000e+10,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [ -3.37500000e+05,   0.00000000e+00,   0.00000000e+00,
           3.39581250e+05,   0.00000000e+00,   0.00000000e+00,
           8.32500000e+06,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,  -2.08125000e+03,   0.000

In [7]:
@extend
class Frame2D:
    
    def buildP(self,loadcase='all'):
        P = np.mat(np.zeros((self.ndof,1)))
        for node,load,factor in self.iter_nodeloads(loadcase):
            P[node.dofnums] += load.forces * factor
        for node,load,factor in self.iter_nodedeltas(loadcase):
            P[node.dofnums] += load.forces * factor
        return P

In [8]:
##test:
f.buildP('all')

matrix([[ -2.00000000e+05],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [ -1.00000000e+01],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00]])

In [9]:
@extend
class Frame2D:
    
    def calc_fefs(self,loadcase='all'):
        ll = defaultdict(list)
        for memb,load,factor in self.iter_memberloads(loadcase):
            ll[memb].append((load,factor))
        fef = {memb:memb.fefs(loads) for memb,loads in ll.items()}
        #self.loadcase_fefs[loadcase] = fef
        return fef    

    def buildMP(self,memb_fefs):
        MP = np.mat(np.zeros((self.ndof,1)))
        for memb,mfefs in memb_fefs.items():
            gfefs = memb.Tm.T * mfefs.fefs
            dofnums = memb.nodej.dofnums + memb.nodek.dofnums
            MP[dofnums] -= gfefs
        return MP

In [10]:
class ResultSet(object):
    
    """Instances of class ResultSet gather together all of the results from
    one complete structural analysis of one load case."""
    
    def __init__(self,loadcase):
        self.loadcase = loadcase
        self.node_P = None       # applied node loads
        self.memb_P = None       # applied fixed end member forces
        self.memb_fefs = {}      # fixed end member forces, indexed by member ID
        self.node_displacements = None  # unconstrained node displacements
        self.reaction_displacements = None # constrained node displacements
        self.reaction_forces = None        # constrained node reactions

In [11]:
@extend
class Frame2D:
    
    def solve(self,loadcase='all'):
        
        self.number_dofs()
        K = self.buildK()

        rs = ResultSet(loadcase)
        rs.memb_fefs = self.calc_fefs(loadcase)
        P = rs.node_P = self.buildP(loadcase)
        MP = rs.memb_P = self.buildMP(rs.memb_fefs)
        P = P + MP
        
        D = np.mat(np.zeros((self.ndof,1)))
        
        N = self.nfree
        Kff = K[:N,:N]  # partition the matrices ....
        Kfc = K[:N,N:]
        Kcf = K[N:,:N]
        Kcc = K[N:,N:]
        
        D[:N] = np.linalg.solve(Kff,P[:N] - Kfc*D[N:])  # displacements
        R = Kcf*D[:N] + Kcc*D[N:] - P[N:]    # reactions at the constrained DOFs
        rs.node_displacements = D
        rs.reaction_forces = R
        return rs

In [12]:
##test:
f.reset()
f.install_all()
rs = f.solve('one')
f.print_input()


Frame frame-1:


              # of nodal degrees of freedom: 12
  # of constrained nodal degrees of freedom: 5
# of unconstrained nodal degrees of freedom: 7  (= degree of kinematic indeterminacy)

                               # of members: 3
                             # of reactions: 5
                                 # of nodes: 4
                            # of conditions: 2
           degree of statical indeterminacy: 0



Nodes:

Node          X         Y  Constraints  DOF #s
----      -----     -----  -----------  ------
A             0         0  FX,FY,MZ     7,8,9
B             0      4000               0,1,2
C          8000      4000               3,4,5
D          8000         0  FX,FY        10,11,6



Members:

Member   Node-J  Node-K    Length       dcx       dcy  Size                Ix           A  Releases
------   ------  ------    ------   -------   -------  --------      --------       -----  --------
AB       A       B         4000.0   0.00000   1.00000  W310x9

In [13]:
##test:
rs.node_displacements

matrix([[ -1.68168168e+02],
        [ -6.70731707e-01],
        [ -2.69747726e-02],
        [ -1.68168168e+02],
        [ -7.92682927e-01],
        [  2.88653913e-02],
        [  4.20420420e-02],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00]])

In [14]:
##test:
rs.reaction_forces

matrix([[  3.50000000e+05],
        [  4.12500000e+05],
        [ -1.40000000e+09],
        [ -5.82076609e-11],
        [  4.87500000e+05]])

#### Compare
Compare computed reactions with correct reactions.

In [15]:
##test:
# here are the correct reactions:
R = rs.reaction_forces
CR = np.matrix([350E3,412.5E3,-1400E6,0,487.5E3]).T
(R-CR)/CR

matrix([[  5.32184328e-15],
        [  5.88426536e-14],
        [  5.44956752e-15],
        [            -inf],
        [  0.00000000e+00]])

Comparison is good.

In [16]:
##test:
f.print_node_displacements(rs.node_displacements)


Node Displacements:

Node        DX         DY      Rotation
----      ------     ------   ---------
A          0.000      0.000   0.0000000
B       -168.168     -0.671  -0.0269748
C       -168.168     -0.793   0.0288654
D          0.000      0.000   0.0420420


In [17]:
##test:
f.print_reactions(rs.reaction_forces)


Reactions:

Node        FX         FY         MZ  
----     -------    -------    -------
A        350.000    412.500  -1400.000
D         -0.000    487.500      --   
